In [ ]:
pip install scanpy

In [ ]:
pip install besca

In [ ]:
import numpy as np ## numpy is a library for mathematical functions and numerics
import pandas as pd ## pandas is a library for working with data frames
import scanpy as sc ## scanpy allows us working with single-cell omics data. besca is built upon scanpy
import matplotlib.pyplot as plt ## matplotlib offers functions for visualizations
import scipy
from scipy import sparse, io ## scipy provides algorithms for optimization, integration, and other numerical operations
import besca as bc ## besca enables reproducible single-cell analysis for translational studies
import seaborn as sns ## seaborn is a high-level plotting library
import os ## standard library to interact with the operational system
import glob


In [ ]:
path = ''
filename = 'CRC_Korean.annotated.updated.h5ad'
adata = sc.read(path + filename)

healthy = adata[adata.obs.Patient == 'SMC01']
healthy = healthy[healthy.obs.dblabel == 'central memory CD4-positive, alpha-beta T cell']
h = healthy.X

cancer = adata[adata.obs.Patient == 'SMC07']
cancer = cancer[cancer.obs.dblabel == 'central memory CD4-positive, alpha-beta T cell']
c = cancer.X


#to slice matrices to first 100 genes
h = h[:, 0:100]
c = c[:, 0:100]

In [ ]:
adata.obs.dblabel.unique() #list of cell types present in data

In [ ]:
#to slice matrices to first 100 genes
h = h[:, 0:100]
c = c[:, 0:100]

In [ ]:
# Calculate gene correlation matrix for cells in healthy and cancer patients
corr_c = pd.DataFrame(c).corr(method='spearman')
corr_h = pd.DataFrame(h).corr(method='spearman')

# change row indeces to gene names
corr_c.index = list(cancer.var_names[:100])
corr_h.index = list(healthy.var_names[:100])

In [ ]:
# Distance correlation matrix, heatmap & hist for c
import dcor

dc_c = np.empty((c.shape[1], c.shape[1]))
for i in range(c.shape[1]):
	for j in range(c.shape[1]):
		dc_c[i,j] = dcor.distance_correlation(c[:,i], c[:,j])
	print(i)

sns.heatmap(dis_c, annot=False)

dis_c = 1 - abs(dc_c/np.max(dc_c))
dis_c = dis_c - np.identity(c.shape[1])*dis_c[1,1]

hist_c = plt.hist(dc_c, bins=40)
plt.title("Histogram")
plt.show()

In [ ]:
all(corr_c.index == corr_h.index)

In [ ]:
# Correlation heatmap
plt.figure(figsize = (15,10))
sns.heatmap(corr_c, annot=False)

In [ ]:
# histogram plot
hist_c = plt.hist(corr_c, bins=40)
plt.title("Histogram")
plt.show()

In [ ]:
#kmeans example code
from sklearn.cluster import KMeans
n_cl = 5
label_c = KMeans(n_clusters=n_cl, random_state=0).fit_predict(corr_c)
label_h = KMeans(n_clusters=n_cl, random_state=0).fit_predict(corr_h)


In [ ]:
# Silhouette plot for c
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

range_n_clusters = range(1,10)
for n_clusters in range_n_clusters:
	cluster_labels = KMeans(n_clusters=n_clusters, random_state=0).fit_predict(dis_c)
	silhouette_avg = silhouette_score(dis_c, cluster_labels, metric="precomputed")
	print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )


In [ ]:
# Sankey diagram
import plotly.graph_objects as go
from plotly.offline import plot

mc = max(label_c+1)
mh = max(label_h+1)

lab_c = ['C_{}'.format(x) for x in range(0,mc)]
lab_h = ['H_{}'.format(x) for x in range(0,mh)]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = lab_h + lab_c,
      color = "blue"
    ),
    link = dict(
      source = label_h,
      target = label_c + mh + 1,
      value =  [1] * len(label_c)
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)

plot(fig, filename='basic-heatmap.html')

In [ ]:
# find genes, which are not in the same clusters
count_c = np.bincount(label_c)
count_h = np.bincount(label_h)

ind_c = sorted(range(len(count_c)), key=lambda k: count_c[k])
ind_h = sorted(range(len(count_h)), key=lambda k: count_h[k])

mapping_c = {i : 0 for i in ind_c}
mapping_h = {i : 0 for i in ind_h}

indx = list(mapping_c.keys())
for i in range(n_cl):
    mapping_c[indx[i]] = i
indx = list(mapping_h.keys())
for i in range(n_cl):
    mapping_h[indx[i]] = i

lab_c_sort = np.array([mapping_c.get(number, number) for number in label_c])
lab_h_sort = np.array([mapping_h.get(number, number) for number in label_h])

equal_indces = lab_c_sort == lab_h_sort
true_indces = np.sum(equal_indces)

unequal_indces = np.array([not elem for elem in equal_indces])
print(corr_c.index[unequal_indces])
# print(corr_h.index[unequal_indces])
print(np.sum(unequal_indces))

In [ ]:
### NEW CODE FOR SUBSETTING BASED ON DGE

In [ ]:
# select a Tregs
tregs = adata[adata.obs.dblabel == 'regulatory T cell']

In [ ]:
# find differentially expressed genes in the cell subset
sc.tl.rank_genes_groups(tregs, groupby='Tissue', method='wilcoxon')

# make visualization
# sc.pl.rank_genes_groups_matrixplot(tregs, n_genes=8)

In [ ]:
from itertools import chain

# subset based top 20 DE genes
ls = tregs.uns['rank_genes_groups']['names'][:20].tolist()
deg_ls = list(chain.from_iterable(ls))

lst = list(range(len(tregs.raw.var.SYMBOL)))
df = pd.DataFrame()
df['idxs'] = lst
df['SYMBOL'] = tregs.raw.var.SYMBOL.tolist()

subset_df = df[df['SYMBOL'].isin(deg_ls)]

subset_df['idxs'].tolist()
indices = df[df['SYMBOL'].isin(deg_ls)]['idxs'].tolist()

tregs_N = tregs[tregs.obs.Tissue == 'Normal']
tregs_T = tregs[tregs.obs.Tissue == 'Tumor']

# reduced gene expression matrix
reduced_matrix_N = tregs_N.raw.X[:,indices].todense()
reduced_matrix_T = tregs_T.raw.X[:,indices].todense()

# add gene names as row names
reduced_matrix_N.indices = subset_df.SYMBOL
reduced_matrix_T.indices = subset_df.SYMBOL

In [ ]:
# calculate correlation matrix Normal
corr_reduced_N = pd.DataFrame(reduced_matrix_N).corr(method='pearson')
corr_reduced_N.index = subset_df['SYMBOL']

# calculate correlation matrix Normal
corr_reduced_T = pd.DataFrame(reduced_matrix_T).corr(method='pearson')
corr_reduced_T.index = subset_df['SYMBOL']

In [ ]:
# Correlation heatmap
plt.figure(figsize = (15,10))
plt.title("Normal")
plt.xlabel('Years')
sns.heatmap(corr_reduced_N, annot=False)
plt.figure(figsize = (15,10))
plt.title("Tumor")
sns.heatmap(corr_reduced_T, annot=False)